In [2]:
# %load model.py
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

Using TensorFlow backend.


In [ ]:
    # input_size = (행, 렬, 채널)
def unet(pretrained_weights = None,input_size = (256,256,1)):
    #Conv2D 인자(필터 수, kernel size, 활성화 함수, 경계 처리 방법, 가중치 초기화 방법)
    # padding(경계 처리 방법) : 1)'valid' 유효한 영역만 출력. 따라서 출력 이미지 사이즈는 입력 사이즈보다 작음.
    #                           2)'same' 출력 이미지 사이즈가 입력 이미지 사이즈와 동일함.
    # 가중치 초기화 방법 'he_normal' : ReLU를 활성화 함수로 사용 시 Xavier 초기값 설정이 비효율적인 결과를 보이는 것을 확인했는데, 이런 경우 사용하는 초기화 방법
    #MaxPooling2D(수직, 수평 축소 비율 지정 : (2,2)이면 출력 영상 크기는 입력 영상 크기의 반으로 줄어듬)
    #Dropout(드롭아웃 비율) : 과적합을 줄이기 위해 일정 비율로만 학습시키는 것
    #concatenate(텐서, 축) : x = Concatenate()([x1, x2]), Concatenate는 레이어를 길게 이어 붙임.
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    # 모델 컴파일 : 훈련전 추가적인 설정단계
    # 1) 최적화(Optimizer) : 데이터와 손실 함수를 바탕으로 모델의 업데이트 방법을 결정
    # Adam(Adaptive Moment Estimation) : 경사하강법 중 RMSProp과 Momentum 방식을 합친 알고리즘. objective function을 사용한다 하더라도 안정적으로 최적화를 위한 하강이 가능. 과거 gradient 크기를 참고하여 적용함.
    # 2) 손실함수(Loss) : 훈련하는 동안 모델의 오차를 측정. 올바른 학습이 진행되도록 손실함수를 최소화해야함
    # binary_crossentropy : 카테고리가 2개인 경우, 교차 엔트로피는 주로 분류 문제에서 많이 사용되는데, 특별히 예측 값이 참과 거짓 둘 중 하나인 형식일 때는 binary_crossentropy(이항 교차 엔트로피)를 씀.
    # 3) 지표(Metrics) : 훈련과 테스트 단계를 모니터링하기 위해 사용.
    # ['accuracy'] : 올바르게 분류된 이미지의 비율인 정확도를 사용
    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    #pretrained_weights 있다면 로드
    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model